# Análise de gastos da UFCG em relação ao seu gasto

## Importando bibliotecas e preparando dados

In [3]:
# Importações
import pandas as pd
import seaborn as sns

In [10]:
orcamento_completo = pd.read_csv('../consulta_siop_orcamento_ufcg.csv')
orcamento_objetivo = orcamento_completo.iloc[:, 18:]
orcamento_objetivo['ano'] = orcamento_completo['ano']
orcamento_objetivo

,objetivo,projeto_de_lei,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,ano
0,[valor não detalhado],0,7.686.431,14.586.431,0,0,0,2012
1,[valor não detalhado],0,22.956.997,22.956.997,0,0,0,2012
2,[valor não detalhado],0,38.666.570,38.666.570,0,0,0,2012
3,[valor não detalhado],21.584.508,0,0,0,0,0,2012
4,[valor não detalhado],38.666.570,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...
4958,[valor não detalhado],0,13.000,13.000,0,0,0,2023
4959,[valor não detalhado],13.000,0,0,0,0,0,2023
4960,[valor não detalhado],0,68.000,68.000,0,0,0,2023
4961,[valor não detalhado],68.000,0,0,0,0,0,2023


## Tratando os dados

In [13]:
objetivo_copia: pd.DataFrame = orcamento_objetivo.copy(deep=True)

for coluna in objetivo_copia.columns[1:-1]:
    objetivo_copia[coluna] = objetivo_copia[coluna].map(lambda value: value.replace(".",""))
    objetivo_copia[coluna] = objetivo_copia[coluna].astype('int64')

objetivo_copia


,objetivo,projeto_de_lei,dotacao_inicial,dotacao_atual,empenhado,liquidado,pago,ano
0,[valor não detalhado],0,7686431,14586431,0,0,0,2012
1,[valor não detalhado],0,22956997,22956997,0,0,0,2012
2,[valor não detalhado],0,38666570,38666570,0,0,0,2012
3,[valor não detalhado],21584508,0,0,0,0,0,2012
4,[valor não detalhado],38666570,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...
4958,[valor não detalhado],0,13000,13000,0,0,0,2023
4959,[valor não detalhado],13000,0,0,0,0,0,2023
4960,[valor não detalhado],0,68000,68000,0,0,0,2023
4961,[valor não detalhado],68000,0,0,0,0,0,2023


In [14]:
objetivo_informacoes = objetivo_copia.groupby(['objetivo', 'ano'])[['projeto_de_lei', 
                                                           'dotacao_inicial', 
                                                           'dotacao_atual', 
                                                           'empenhado', 
                                                           'liquidado', 
                                                           'pago']].agg([
    sum
])

objetivo_informacoes

projeto_de_lei  \
                                                                   sum   
objetivo                                           ano                   
0582 - Expandir, interiorizar, democratizar e q... 2014        1598204   
                                                   2015        1511577   
0597 - Promover, em articulação com os sistemas... 2012         229892   
                                                   2013        1360240   
                                                   2014        1704881   
                                                   2015         642701   
0803 - Apoiar a formação de pessoal qualificado... 2012         461369   
                                                   2013        2324165   
                                                   2014        2011276   
                                                   2015        1252583   
0841 - Ampliar o acesso à educação superior com... 2012       74484103   
                                                   2013       84182086   
                                                   2014       93645202   
                                                   2015      107301602   
1009 - Ampliar o acesso à educação profissional... 2016        1903044   
                                                   2017        1541230   
                                                   2018        1229204   
                                                   2019        1659645   
1010 - Ampliar o acesso à educação superior de ... 2016      110671031   
                                                   2017      102848784   
                                                   2018       87532645   
                                                   2019       89416257   
1204 - Ampliar o acesso à educação profissional... 2020         982507   
                                                   2021         959417   
                                                   2022        1704567   
                                                   2023        1661362   
1237 - Fomentar a formação de pessoal qualifica... 2020       55953608   
                                                   2021       32587658   
                                                   2022       77579609   
                                                   2023       72887646   
[valor não detalhado]                              2012      304127438   
                                                   2013      315468518   
                                                   2014      381347718   
                                                   2015      422531400   
                                                   2016      478443148   
                                                   2017      542525837   
                                                   2018      564164581   
                                                   2019      591671792   
                                                   2020      578404832   
                                                   2021      366415747   
                                                   2022      664228115   
                                                   2023      679352320   

                                                        dotacao_inicial  \
                                                                    sum   
objetivo                                           ano                    
0582 - Expandir, interiorizar, democratizar e q... 2014         1598204   
                                                   2015         1511577   
0597 - Promover, em articulação com os sistemas... 2012          229892   
                                                   2013         1360240   
                                                   2014         1104881   
                                                   2015          642701   
0803 - Apoiar a formação de pessoal qualificado...

## Interpretando os dados

O Data Frame acima será fundamental para a análise dos dados, tendo as informações essenciais para o estudo dos objetivos de gastos da UFCG.

Informações imediatas que tiramos a partir dos índices do Data Frame:

    * Os objetivos não ocorreram em todos os anos, existindo anos específicos em que ocorreram determinados objetivos de gasto.
    * A única repetição que ocorreu em todos os anos foram os objetivos não especificados